In [1]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from keras_preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [21]:
train_head()

NameError: name 'train_head' is not defined

In [22]:
# converting to (toxic or not label)
combine = train[["toxic","severe_toxic","obscene","threat","insult","identity_hate"]].sum(axis=1)
y = (combine != 0).astype(int)
y = y.values
X = train['comment_text']

In [29]:
max_features = 20000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X))
list_tokenized_train = tokenizer.texts_to_sequences(X)

In [31]:
maxlen = 200
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)

In [33]:
# model 
inp = Input(shape=(maxlen, ))
embed_size = 128
x = Embedding(max_features, embed_size)(inp)
x = LSTM(60, return_sequences=True,name='lstm_layer')(x)
x = GlobalMaxPool1D()(x)
x = Dropout(0.1)(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)

In [34]:
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

In [35]:
batch_size = 32
epochs = 2
model.fit(X_t,y, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/2
4488/4488 [==============================] - 618s 137ms/step - loss: 0.1291 - accuracy: 0.9550 - val_loss: 0.1016 - val_accuracy: 0.9621
Epoch 2/2
4488/4488 [==============================] - 644s 144ms/step - loss: 0.0856 - accuracy: 0.9683 - val_loss: 0.1006 - val_accuracy: 0.9632


In [36]:
import pickle

model.save('english_model.h5',save_format='h5')


# saving
with open('english_tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
